<a href="https://colab.research.google.com/github/CarolineWitts/hello-word/blob/main/Aprendizagem_por_refor%C3%A7o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Aprendizagem por reforço com Q-Learning

In [27]:
import gym 
import random

In [28]:
env = gym.make('Taxi-v3').env

In [29]:
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [30]:
env.reset()
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [31]:
env.action_space

Discrete(6)

In [32]:
len(env.P)

500

In [33]:
env.P[484]

{0: [(1.0, 484, -1, False)],
 1: [(1.0, 384, -1, False)],
 2: [(1.0, 484, -1, False)],
 3: [(1.0, 464, -1, False)],
 4: [(1.0, 484, -10, False)],
 5: [(1.0, 484, -10, False)]}

In [34]:
import numpy as np

In [35]:
q_table

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [36]:
random.uniform(0,1)

0.4232535034184399

In [37]:
q_table = np.zeros([env.observation_space.n, env.action_space.n])
q_table.shape

(500, 6)

In [43]:
%%time
from IPython.display import clear_output

alpha= 0.1
gama= 0.6
epsilon = 0.1

for i in range(100000):
  estado = env.reset()

  penalidades, recompensa = 0, 0
  done=False
  while not done:
    #Exploração do ambiente, buscar o melhor valor 
    if random.uniform(0, 1) < epsilon:
      acao = env.action_space.sample()
      #Exploitation sempre seguir para melhor
    else:
      acao = np.argmax(q_table[estado])

      proximo_estado, recompensa, done, info = env.step(acao)

      q_antigo = q_table[estado, acao]
      proximo_maximo = np.max(q_table[proximo_estado])

      q_novo = (1 - alpha) * q_antigo + alpha * (recompensa + gamma * proximo_maximo)
      q_table[estado, acao] = q_novo


      if recompensa == -10:
        penalidades +=1
    
    estado = proximo_estado 

  if i % 100 == 0:
    clear_output(wait=True)
    print('Episodio',i)

print('treinamento concluido')

Episodio 99900
treinamento concluido
CPU times: user 58.7 s, sys: 21.4 s, total: 1min 20s
Wall time: 1min 1s


In [52]:
q_table[223]

array([ -2.47061241,  -2.42370134,  -2.47061275,  -2.42373085,
       -11.45101454, -11.45096997])

0 = south 1 =  north 2= east, 3 = west, 4 = pickup, 5 = dropoff

In [49]:
env.reset()
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [51]:
env.encode(2, 1, 0, 3)

223

In [61]:
total_penalidades = 0
episodios = 50
frames = []

for _ in range(episodios):
  estado = env.reset()
  penalidade, recompensa = 0, 0
  done = False
  while not done:
    acao = np.argmax(q_table[estado])
    estado, recompensa, done, info = env.step(acao)

    if recompensa == -10:
      penalidades += 1

    frames.append({
        'frame': env.render(mode='ansi'),
        'state': estado,
        'action': acao,
        'reward': recompensa

    })

  total_penalidades += penalidades

print('Episódios', episodios)
print('Penalidades', total_penalidades)

Episódios 50
Penalidades 0


In [62]:
frames

[{'action': 2,
  'frame': '+---------+\n|R: | : :\x1bG\x1b|\n| : | : : |\n| : : :\x1b \x1b: |\n| | : | : |\n|Y| : |\x1bB\x1b: |\n+---------+\n  (East)\n',
  'reward': -1,
  'state': 273},
 {'action': 0,
  'frame': '+---------+\n|R: | : :\x1bG\x1b|\n| : | : : |\n| : : : : |\n| | : |\x1b \x1b: |\n|Y| : |\x1bB\x1b: |\n+---------+\n  (South)\n',
  'reward': -1,
  'state': 373},
 {'action': 0,
  'frame': '+---------+\n|R: | : :\x1bG\x1b|\n| : | : : |\n| : : : : |\n| | : | : |\n|Y| : |\x1b\x1bB\x1b\x1b: |\n+---------+\n  (South)\n',
  'reward': -1,
  'state': 473},
 {'action': 4,
  'frame': '+---------+\n|R: | : :\x1bG\x1b|\n| : | : : |\n| : : : : |\n| | : | : |\n|Y| : |\x1bB\x1b: |\n+---------+\n  (Pickup)\n',
  'reward': -1,
  'state': 477},
 {'action': 1,
  'frame': '+---------+\n|R: | : :\x1bG\x1b|\n| : | : : |\n| : : : : |\n| | : |\x1b_\x1b: |\n|Y| : |B: |\n+---------+\n  (North)\n',
  'reward': -1,
  'state': 377},
 {'action': 1,
  'frame': '+---------+\n|R: | : :\x1bG\x1b|\n| : | : : 

In [64]:
from time import sleep
for frame in frames:
  clear_output(wait=True)
  print(frame['frame'])
  print('Estado', frame['state'])
  print('Ação', frame['action'])
  print('Recompensa', frame['reward'])
  sleep(.5)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Estado 410
Ação 5
Recompensa 20
